In [134]:
import pandas as pd
import cPickle as pickle
import numpy as np
from collections import defaultdict
from scipy import stats

In [5]:
ls ../data/draft_stats/

drafts_677.csv   drafts_680.csv   drafts_682.csv   drafts_685.csv
drafts_678.csv   drafts_681.csv   drafts_683.csv
drafts_679.csv   drafts_681b.csv  drafts_684.csv


In [85]:
df_677 = pd.read_csv('../data/draft_stats/drafts_677.csv')

In [100]:
df_677.loc[df_677[df_677['Hero']=='furion'].index[0], 'Name'] = "Nature's Prophet"

In [140]:
df_677.ix[12]['Name']

"Nature's Prophet"

In [27]:
df_677.sort('Name').head()

,Hero,Name,Pick %,Ban %,Pick or Ban %,Times Picked,Times Banned,Times Picked or Banned
30,alchemist,Alchemist,9.4,10.1,19.5,163,175,338
62,ancient_apparition,Ancient Apparition,2.7,1.3,4.0,46,23,69
19,antimage,Anti Mage,15.2,18.9,34.1,264,328,592
78,axe,Axe,1.0,0.1,1.1,17,2,19
17,bane,Bane,22.6,17.5,40.1,392,304,696


In [141]:
all_draft_dfs = {}
for i in xrange(677, 687):
    df = pd.read_csv('../data/draft_stats/drafts_{0}.csv'.format(i))
    df.loc[df[df['Hero']=='furion'].index[0], 'Name'] = "Nature's Prophet" # fix apostrophe
    df.loc[df[df['Hero']=='antimage'].index[0], 'Name'] = "Anti-Mage" # fix hyphen
    df.drop(['Hero', 'Times Picked', 'Times Banned'], axis=1, inplace=True)
    df.columns = ['hero', 'pick%', 'ban%', 'pb%', 'times_pb']
    all_draft_dfs[i] = df.copy()

In [142]:
for patch_num, df in all_draft_dfs.iteritems():
    if patch_num < 686:
        next_df = all_draft_dfs[patch_num+1]
        next_pbs = dict(next_df[['hero', 'pb%']].values)
        new_pb_list = []
        for hero in df['hero']:
            if hero in next_pbs:
                new_pb_list.append(next_pbs[hero])
            else:
                new_pb_list.append(None)
        df['next_patch_pb%'] = new_pb_list
    else:
        df['next_patch_pb%'] = [None]*len(df)
        
    if patch_num > 677:
        prev_df = all_draft_dfs[patch_num-1]
        prev_pbs = dict(prev_df[['hero', 'pb%']].values)
        old_pb_list = []
        for hero in df['hero']:
            if hero in prev_pbs:
                old_pb_list.append(prev_pbs[hero])
            else:
                old_pb_list.append(None)
        df['prev_patch_pb%'] = old_pb_list
    else:
        df['prev_patch_pb%'] = [None]*len(df)
        
    df['patch'] = [patch_num]*len(df)

In [144]:
pd.concat(all_draft_dfs).head(3)

hero  pick%  ban%   pb%  times_pb next_patch_pb%  \
677 0      Batrider   24.0  76.0  99.9      1734           99.7   
    1  Nyx Assassin   33.7  61.7  95.3      1654             64   
    2        Magnus   49.1  41.2  90.3      1566           16.1   

      prev_patch_pb%  patch  
677 0           None    677  
    1           None    677  
    2           None    677

In [147]:
max(all_draft_dfs.keys())

686

In [3]:
ls ../data/hero_stats

heroes_677.csv   heroes_680.csv   heroes_682.csv   heroes_685.csv
heroes_678.csv   heroes_681.csv   heroes_683.csv   heroes_686.csv
heroes_679.csv   heroes_681b.csv  heroes_684.csv


In [5]:
heroes_677 = pd.read_csv('../data/hero_stats/heroes_677.csv')

In [6]:
heroes_677.head()

,Hero,Name,Games,W,L,Win %,L.1,K,D,A,Tot. Gold,LH,Denies,GPM,XPM
0,alchemist,Alchemist,176,91,85,0.517,17.6,4.7,4.1,8.9,19947.0,189.3,10.3,540.8,496.8
1,ancient_apparition,Ancient Apparition,49,19,30,0.388,10.7,2.4,5.7,8.1,6155.6,22.6,3.5,182.4,206.7
2,antimage,Anti Mage,282,144,138,0.511,20.1,6.6,2.9,4.8,21876.7,304.3,19.6,584.1,625.0
3,axe,Axe,19,8,11,0.421,13.6,5.4,5.1,5.9,9916.4,100.0,3.1,331.8,369.2
4,bane,Bane,431,212,219,0.492,12.4,2.8,5.1,8.0,7355.6,27.0,3.9,209.9,259.9


In [82]:
hero_patch_wrs = {}
for i in xrange(677, 687):
    df = pd.read_csv('../data/hero_stats/heroes_{0}.csv'.format(i))
    df.loc[df[df['Hero']=='furion'].index[0], 'Name'] = "Nature's Prophet" # fix apostrophe
    df.loc[df[df['Hero']=='antimage'].index[0], 'Name'] = "Anti-Mage" # fix hyphen
    hero_patch_wrs[i] = df[['Name', 'W', 'L', 'Win %']].set_index('Name')

In [131]:
hero_patch_wrs[677][hero_patch_wrs[677]['W']>20].sort('Win %', ascending=False).head()

,W,L,Win %
Name,,,
Io,179,123,0.593
Chen,178,126,0.586
Dragon Knight,97,69,0.584
Nature's Prophet,314,243,0.564
Clockwerk,255,205,0.554


In [136]:
pairs_677 = pd.read_csv('../data/pair_stats/pairs_677.csv')

In [137]:
hero_pairs_by_patch = {}
for i in xrange(677, 687):
    df = pd.read_csv('../data/pair_stats/pairs_{0}.csv'.format(i))
    hero_pairs_by_patch[i] = df

In [139]:
def check_sig(pair_wins, pair_losses, hero, hero_wr_df, threshold=0.1):
    solo_wins = hero_wr_df.loc[hero]['W']
    solo_losses = hero_wr_df.loc[hero]['L']
    contingency_table = np.array([[pair_wins, pair_losses], [solo_wins, solo_losses]])
    return stats.chi2_contingency(contingency_table)[1] < threshold

In [147]:
hero_ally_dicts = {}
for i in xrange(677, 687):
    pairs_df = hero_pairs_by_patch[i]
    hero_wrs = hero_patch_wrs[i]
    
    hero_allies = defaultdict(list)
    for _, pair in pairs_df.iterrows():
        hero, other, joint_wr = pair['Name'], pair['Name.1'], pair['Win %']
        hero = hero.replace('Natures Prophet', "Nature's Prophet").replace('Anti Mage', 'Anti-Mage')
        other = other.replace('Natures Prophet', "Nature's Prophet").replace('Anti Mage', 'Anti-Mage')
        hero_solo_wr = hero_wrs.loc[hero]['Win %']
        other_solo_wr = hero_wrs.loc[other]['Win %']

        hero_sig = check_sig(pair['W'], pair['L'], hero, hero_wrs)
        other_sig = check_sig(pair['W'], pair['L'], other, hero_wrs)

        hero_allies[hero].append((other, round(joint_wr - hero_solo_wr, 4), hero_sig))
        hero_allies[other].append((hero, round(joint_wr - other_solo_wr, 4), other_sig)) 

    hero_ally_dicts[i] = hero_allies

In [156]:
sorted(hero_ally_dicts[684]['Leshrac'], key=lambda x: x[1], reverse=True)[:5]

[('Bounty Hunter', 0.218, True),
 ('Dazzle', 0.18, True),
 ('Shadow Fiend', 0.106, False),
 ('Beastmaster', 0.104, False),
 ('Spirit Breaker', 0.091, False)]